In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))
from utils import de, dcde
from benchmark.simple import rosenbrock, rastrigin, shubert
from IPython.display import clear_output
import time
from prettytable import PrettyTable

In [2]:
runs = 1000

populationSize = 20
boundaries = [(-10,10)]
dimensions = 2
mutationFactor = 0.5
crossingRate = 0.5
fitness = shubert
maxgens = 10e3
stopDiff = 1e-7

In [3]:
dcde2Fitness = np.zeros(runs)
dcdeFitness = np.zeros(runs)
deFitness = np.zeros(runs)

dcde2Time = np.zeros(runs)
dcdeTime = np.zeros(runs)
deTime = np.zeros(runs)

globalStart = time.time()
run = 0
for r in range(runs):

    # regular DE
    deStart = time.time()
    deBest = de.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = maxgens,
        stopDiff = stopDiff,
    )
    deEllapsed = time.time() - deStart

    # divide and conquer DE
    dcdeStart = time.time()
    dcdeBest = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = maxgens,
        stopDiff = stopDiff,
        maxsubgens = 500,
    )
    dcdeEllapsed = time.time() - dcdeStart
    
    # divide and conquer DE + different mutation on conquer step
    dcde2Start = time.time()
    dcde2Best = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor + .25,
        conquerMutationFactor = mutationFactor - .25,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = maxgens,
        stopDiff = stopDiff,
        maxsubgens = 500,
    )
    dcde2Ellapsed = time.time() - dcde2Start
    
    dcde2Fitness[r] = fitness(dcde2Best)
    dcdeFitness[r] = fitness(dcdeBest)
    deFitness[r]   = fitness(deBest)

    dcde2Time[r] = dcde2Ellapsed    
    dcdeTime[r] = dcdeEllapsed
    deTime[r]   = deEllapsed
    
    run = r + 1  
    deMean = np.mean(deFitness[:run])
    deStd = np.std(deFitness[:run])
    deMin = np.min(deFitness[:run])
    deMax = np.max(deFitness[:run])
    deMeanTime = np.mean(deTime[:run])
    
    dcdeMean = np.mean(dcdeFitness[:run])
    dcdeStd = np.std(dcdeFitness[:run])
    dcdeMin = np.min(dcdeFitness[:run])
    dcdeMax = np.max(dcdeFitness[:run])
    dcdeMeanTime = np.mean(dcdeTime[:run])
  
    dcde2Mean = np.mean(dcde2Fitness[:run])
    dcde2Std = np.std(dcde2Fitness[:run])
    dcde2Min = np.min(dcde2Fitness[:run])
    dcde2Max = np.max(dcde2Fitness[:run])
    dcde2MeanTime = np.mean(dcde2Time[:run])

    ellapsedTime = time.time() - globalStart
    remaining = (ellapsedTime/run) * (runs - run)
    ellapsedTimeStr = time.strftime('%H:%M:%S', time.gmtime(ellapsedTime))
    remainingStr = time.strftime('%H:%M:%S', time.gmtime(remaining))
    
    clear_output()
    p = PrettyTable(["", "DE", "DCDE", "DCDE 2"])
    
    p.add_row(["Mean", deMean, dcdeMean, dcde2Mean])
    p.add_row(["Std", deStd, dcdeStd, dcde2Std])
    p.add_row(["Best", deMin, dcdeMin, dcde2Min])
    p.add_row(["Worst", deMax, dcdeMax, dcde2Max])
    p.add_row(["Time (s)", deMeanTime, dcdeMeanTime, dcde2MeanTime])
    print(p)
    print(f'Run {run}, Ellapsed: {ellapsedTimeStr}, Remaining: {remainingStr}')

+----------+---------------------+------------------------+------------------------+
|          |          DE         |          DCDE          |         DCDE 2         |
+----------+---------------------+------------------------+------------------------+
|   Mean   | -188.24357392420205 |  -186.73090883099945   |  -186.73090883102162   |
|   Std    |  3.878334359927333  | 1.0342805369914487e-10 | 1.4168486809718274e-11 |
|   Best   | -205.56440709628717 |  -186.73090883102392   |   -186.730908831024    |
|  Worst   | -186.73090883523892 |   -186.7309088289982   |   -186.7309088307644   |
| Time (s) |  1.4849147548675536 |   0.721404464006424    |   0.7217828855514526   |
+----------+---------------------+------------------------+------------------------+
Run 1000, Ellapsed: 00:48:50, Remaining: 00:00:00


In [2]:
runs = 1000

populationSize = 20
boundaries = [(-1000,1000)]
dimensions = 200
mutationFactor = 0.5
crossingRate = 0.5
fitness = rastrigin
maxgens = 1500

In [3]:
dcde2Fitness = np.zeros(runs)
dcdeFitness = np.zeros(runs)
deFitness = np.zeros(runs)

dcde2Time = np.zeros(runs)
dcdeTime = np.zeros(runs)
deTime = np.zeros(runs)

globalStart = time.time()
run = 0
for r in range(runs):

    # regular DE
    deStart = time.time()
    deBest = de.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = 2*maxgens # double since divide and conquer has extras maxgens on the divide cycle
    )
    deEllapsed = time.time() - deStart

    # divide and conquer DE
    dcdeStart = time.time()
    dcdeBest = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = mutationFactor,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = maxgens,
    )
    dcdeEllapsed = time.time() - dcdeStart
    
    # divide and conquer DE + different mutation on conquer step
    dcde2Start = time.time()
    dcde2Best = dcde.optimize(
        populationSize = populationSize,
        boundaries = boundaries,
        dimensions = dimensions,
        mutationFactor = .75,
        conquerMutationFactor = .25,
        crossingRate = crossingRate,
        fitness = fitness,
        maxgens = maxgens,
    )
    dcde2Ellapsed = time.time() - dcde2Start
    
    dcde2Fitness[r] = fitness(dcde2Best)
    dcdeFitness[r] = fitness(dcdeBest)
    deFitness[r]   = fitness(deBest)

    dcde2Time[r] = dcde2Ellapsed    
    dcdeTime[r] = dcdeEllapsed
    deTime[r]   = deEllapsed
    
    run = r + 1  
    deMean = np.mean(deFitness[:run])
    deStd = np.std(deFitness[:run])
    deMin = np.min(deFitness[:run])
    deMax = np.max(deFitness[:run])
    deMeanTime = np.mean(deTime[:run])
    
    dcdeMean = np.mean(dcdeFitness[:run])
    dcdeStd = np.std(dcdeFitness[:run])
    dcdeMin = np.min(dcdeFitness[:run])
    dcdeMax = np.max(dcdeFitness[:run])
    dcdeMeanTime = np.mean(dcdeTime[:run])
  
    dcde2Mean = np.mean(dcde2Fitness[:run])
    dcde2Std = np.std(dcde2Fitness[:run])
    dcde2Min = np.min(dcde2Fitness[:run])
    dcde2Max = np.max(dcde2Fitness[:run])
    dcde2MeanTime = np.mean(dcde2Time[:run])

    ellapsedTime = time.time() - globalStart
    remaining = (ellapsedTime/run) * (runs - run)
    ellapsedTimeStr = time.strftime('%H:%M:%S', time.gmtime(ellapsedTime))
    remainingStr = time.strftime('%H:%M:%S', time.gmtime(remaining))
    
    clear_output()
    p = PrettyTable(["", "DE", "DCDE", "DCDE 2"])
    
    p.add_row(["Mean", deMean, dcdeMean, dcde2Mean])
    p.add_row(["Std", deStd, dcdeStd, dcde2Std])
    p.add_row(["Best", deMin, dcdeMin, dcde2Min])
    p.add_row(["Worst", deMax, dcdeMax, dcde2Max])
    p.add_row(["Time (s)", deMeanTime, dcdeMeanTime, dcde2MeanTime])
    print(p)
    print(f'Run {run}, Ellapsed: {ellapsedTimeStr}, Remaining: {remainingStr}')

+----------+--------------------+--------------------+--------------------+
|          |         DE         |        DCDE        |       DCDE 2       |
+----------+--------------------+--------------------+--------------------+
|   Mean   | 15571.860233101068 | 1834.8427181134768 | 1795.4590944097135 |
|   Std    | 7531.825534832794  | 50.281310686262486 | 43.285060352851254 |
|   Best   |  6566.24940339536  | 1695.3377152930282 | 1623.2221492154079 |
|  Worst   | 93435.65913105303  | 2042.923103045816  | 1937.7190229103644 |
| Time (s) | 4.033112906217575  | 4.148532928466797  | 4.150287723302841  |
+----------+--------------------+--------------------+--------------------+
Run 1000, Ellapsed: 03:25:33, Remaining: 00:00:00
